In [1]:
import os
import json
import warnings
import re
import codecs
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from functools import reduce
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.utils import shuffle
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set_style('whitegrid')
sns.set_palette('Set2')

Данный блокнот может служить baselin-ом в соревновании https://www.kaggle.com/c/morecomplicatedsentiment:

> К вашей компании пришел заказчик, которому нужно решение задачи анализа тональности отзывов на товары. Заказчик хочет, чтобы вы оценили возможное качество работы такого алгоритма на небольшой тестовой выборке. При этом больше никаких данных вам не предоставляется.

>Вам предстоит посмотреть на предоставленные заказчиком отзывы, собрать похожие отзывы в качестве обучающей выборки, и поэкспериментировать с постановкой задачи (разметкой вашей выборки на позитивные и негативные примеры) так, чтобы результат на примерах заказчика был по возможности получше.

>Обратите внимание, что заказчик предоставил всего 100 примеров в качестве тестовой выборки - ситуация, когда размеченных данных почти нет - вообще очень частая в индустриальном анализе данных. Конечно, эти отзывы можно было бы идеально разметить вручную и получить максимальное качество, но вы сами не заинтересованы в таком подходе, т.к. потом придется и на всех новых примерах демонстрировать заказчику идеальную работу, что, конечно, вряд ли будет по силам алгоритму.

Датасет для данной задачи был собран с Яндекс Маркета (https://market.yandex.ru/catalog--mobilnye-telefony/54726/list).

## Загрузка данных

In [2]:
with codecs.open('morecomplicatedsentiment/test.csv', 'r') as file:
    test_review = file.read()
    parser = BeautifulSoup(test_review, 'html.parser')
    test_reviews = parser.findAll('review')
    test_review_list = [review.text for review in test_reviews]

In [3]:
def extract_info(text, label):
    try:
        if label == 1:
            text = re.findall(r'Достоинства(:.*?)Недостатки:', text)[0]
        else:
            text = re.findall(r'Недостатки(:.*?)Комментарий:', text)[0]
        return re.sub(r'\W', ' ', text)
    except:
        return text

In [4]:
test = pd.DataFrame(test_review_list, columns=['text'])
test.head()

,text
0,"Ужасно слабый аккумулятор, это основной минус ..."
1,ценанадежность-неубиваемостьдолго держит батар...
2,"подробнее в комментариях\nК сожалению, факт по..."
3,я любительница громкой музыки. Тише телефона у...
4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех..."


In [5]:
test.shape

(100, 1)

In [6]:
train = pd.read_json('morecomplicatedsentiment/train.json', orient='records', lines=True).drop_duplicates()

In [7]:
train['label'] = train['rating'].apply(lambda x: int(x == 5))
train['length'] = train['text'].apply(lambda x: len(x))
train['text'] = train.apply(lambda x: extract_info(x['text'], x['label']), axis=1)

In [8]:
train['label'].value_counts()

1    3320
0    1680
Name: label, dtype: int64

Видно, что выборка не сбалансирована.

In [9]:
train.shape

(5000, 4)

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 5004
Data columns (total 4 columns):
text      5000 non-null object
rating    5000 non-null int64
label     5000 non-null int64
length    5000 non-null int64
dtypes: int64(3), object(1)
memory usage: 195.3+ KB


## Подбор классификатора
Так как целевая переменная не сбалансирована, выберем только 1680 положительных отзывов в обучающую выборку.

In [11]:
train = shuffle(pd.concat([train[train['label'] == 0], 
                           train[train['label'] == 1].sort_values('length', ascending=False).head(1680)]), 
                           random_state=777)

In [12]:
X = train['text'].values
y = train['label'].values

In [13]:
X_test = test['text'].values

In [14]:
stop_words = stopwords.words('russian')

In [15]:
train['label'].value_counts()

1    1680
0    1680
Name: label, dtype: int64

In [16]:
def make_pipeline(vectorizer, transformer, classifier):
    return Pipeline([
            ('vectorizer', vectorizer),
            ('transformer', transformer),
            ('classifier', classifier)
        ])

In [17]:
def make_estimator(classifier, params_grid, scorer, data, labels):
    pipeline = make_pipeline(CountVectorizer(), TfidfTransformer(), classifier)
    grid_cv = RandomizedSearchCV(pipeline, params_grid, scoring=scorer, cv=5, 
                                 random_state=777, n_iter=100, verbose=1, n_jobs=-1)
    grid_cv.fit(data, labels)
    return grid_cv

Логистическая регрессия.

In [18]:
score = cross_val_score(make_pipeline(CountVectorizer(), TfidfTransformer(), LogisticRegression(random_state=777, C=50)), X, y, cv=5).mean()
print(f"LogisticRegression - {score}")

LogisticRegression - 0.9261904761904762


Линейный SVM.

In [19]:
score = cross_val_score(make_pipeline(CountVectorizer(), TfidfTransformer(), LinearSVC(random_state=777)), X, y, cv=5).mean()
print(f"LinearSVC - {score}")

LinearSVC - 0.9336309523809524


Видно, что LinearSVC показывает неммного лучшее качество. Попробуем настроить параметры для LinearSVC.

In [20]:
params_grid_vectorizer = {
    'vectorizer__max_df': [0.85, 0.9, 0.95, 1.0],
    'vectorizer__min_df': [1, 10, 20],
    'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6),
                               (2, 2), (2, 3), (2, 4), (3, 3), (3, 4)],
    'vectorizer__stop_words': [stop_words, None]
}
params_grid_transformer = {
    'transformer__norm': ['l1', 'l2'],
    'transformer__smooth_idf': [True, False],
    'transformer__use_idf': [True, False],
    'transformer__sublinear_tf': [True, False]
}
params_grid_lsvc = {
    'classifier__loss': ['hinge', 'squared_hinge'],
    'classifier__max_iter': np.arange(200, 1000, 100),
    'classifier__tol': [1e-5, 1e-4, 1e-3],
    'classifier__C': np.arange(0.5, 1.2, 0.1)
}

In [21]:
%%time
grid_search_lsvc = make_estimator(LinearSVC(random_state=777), 
                                  {**params_grid_vectorizer, **params_grid_transformer, **params_grid_lsvc}, 'accuracy', X, y)
print("LinearSVC:")
print(f"Лучшее качество - {grid_search_lsvc.best_score_}")
print(f"Параметры - {grid_search_lsvc.best_params_}")

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   25.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  3.3min finished


LinearSVC:
Лучшее качество - 0.9348214285714286
Параметры - {'vectorizer__stop_words': ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'э

## Predictions.

3.1 Дефолтная LogisticRegression

In [22]:
clf = make_pipeline(CountVectorizer(), TfidfTransformer(), LogisticRegression(random_state=777)).fit(X, y).predict(X_test)

In [23]:
with open('submission_lin_reg.csv', 'w') as f:
    f.write(pd.DataFrame(pd.Series(map(str, range(0, 100))).str \
                         .cat(map(str, pd.Series(clf).apply(lambda x: 'neg' if x == 0 else 'pos')), sep=','), 
                         columns=['Id,y']).to_csv(sep=' ', index=False))

Она дает результат порядка 0.944

3.2 Настроенный LinearSVC

In [24]:
make_pipeline(CountVectorizer(), TfidfTransformer(), LinearSVC(random_state=777)).fit(X, y).predict(X_test)

array([0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0])

In [25]:
svc = grid_search_lsvc.best_estimator_
svc.steps

[('vectorizer',
  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                  dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                  lowercase=True, max_df=0.9, max_features=None, min_df=1,
                  ngram_range=(1, 1), preprocessor=None,
                  stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с',
                              'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
                              'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы',
                              'по', 'только', 'ее', 'мне', ...],
                  strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                  tokenizer=None, vocabulary=None)),
 ('transformer',
  TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=True, use_idf=True)),
 ('classifier',
  LinearSVC(C=0.6, class_weight=None, dual=True, fit_intercept=True,
            intercept_scaling=1, loss='hinge', max_iter=300, multi_class='ovr'

In [26]:
pred = svc.predict(X_test)
pred

array([0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0])

In [27]:
with open('submission_lin_svc.csv', 'w') as f:
    f.write(pd.DataFrame(pd.Series(map(str, range(0, 100))).str \
                         .cat(map(str, pd.Series(pred).apply(lambda x: 'neg' if x == 0 else 'pos')), sep=','), 
                         columns=['Id,y']).to_csv(sep=' ', index=False))

Здесь уже результат получается порядка 0.955